In [1]:
import numpy as np 
import mmcv
from PIL  import Image
 
img_a_mask= Image.open('/Users/johnny/Code/openself3d/data/2018-04-16-14-25-19/processed/image_masks/000008_mask.png')
img_a_mask = np.asarray(img_a_mask)
img_a_depth= Image.open('/Users/johnny/Code/openself3d/data/2018-04-16-14-25-19/processed/images/000008_depth.png')
img_a_depth = np.asarray(img_a_depth)

mask_indices = np.where(img_a_mask>0,1,0)
depth_indices = np.where(img_a_depth>0,1,0)
select_indices = mask_indices&depth_indices
img_a_indices = np.where(select_indices>0)
rand_indices = np.random.randint(0,len(img_a_indices[0]),[10000])
u_vec = img_a_indices[0][rand_indices]
v_vec = img_a_indices[1][rand_indices]
uv_vec = (u_vec, v_vec)
img_a_depth_float = img_a_depth.astype(np.float)
DEPTH_IM_SCALE = 1000.0 #将厘米转换为米。
depth_vec_float = img_a_depth_float[uv_vec]*1.0/DEPTH_IM_SCALE
u_vec_float = u_vec.astype(np.float)*depth_vec_float
v_vec_float = v_vec.astype(np.float)*depth_vec_float
print(u_vec_float.size,v_vec_float.size,depth_vec_float.size)
#print(u_vec_float,v_vec_float, depth_vec_float)

z_vec_float = depth_vec_float
full_vec_float = np.vstack((u_vec_float,v_vec_float, z_vec_float))
print(full_vec_float)
print(full_vec_float.shape)



No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
10000 10000 10000
[[190.344 229.599 188.467 ... 158.95  232.165 285.74 ]
 [254.616 220.92  243.608 ... 247.35  243.183 215.875]
 [  0.824   0.789   0.823 ...   0.85    0.787   0.785]]
(3, 10000)


In [2]:
import numpy as np

body_to_rdf = np.zeros((3,3))
body_to_rdf[0, 1] = -1.0
body_to_rdf[1, 2] = -1.0
body_to_rdf[2, 0] = 1.0

rdf_to_body = np.linalg.inv(body_to_rdf)
K = np.array([[1,0,0],
              [0,1,0],
              [0,0,1]
              ])
K[0,0] = 533.6422696034836 # focal x
K[1,1] = 534.7824445233571 # focal y
K[0,2] = 319.4091030774892 # principal point x
K[1,2] = 236.4374299691866 # principal point y
K[2,2] = 1.0
K_inv = np.linalg.inv(K)
point_camera_frame_rdf_vec = np.matmul(K_inv,full_vec_float)

In [3]:
import utils
import mmcv 
from correspondence_finder import apply_transform, invert_transform

scene_pose_data = mmcv.load('/Users/johnny/Code/openself3d/data/2018-04-16-14-25-19/processed/images/pose_data.yaml')
a_pose_data = scene_pose_data[8]['camera_to_world']
img_a_pose = utils.homogenous_transform_from_dict(a_pose_data)
b_pose_data = scene_pose_data[163]['camera_to_world']
img_b_pose = utils.homogenous_transform_from_dict(b_pose_data)

point_world_frame_rdf_vec = apply_transform(point_camera_frame_rdf_vec, img_a_pose)
point_camera_2_frame_rdf_vec = apply_transform(point_world_frame_rdf_vec,invert_transform(img_b_pose))

########### vec2#########
vec2_vec = np.matmul(K, point_camera_2_frame_rdf_vec)
u2_vec = vec2_vec[0]/vec2_vec[2]
v2_vec = vec2_vec[1]/vec2_vec[2]

maybe_z2_vec = point_camera_2_frame_rdf_vec[2]
z2_vec = vec2_vec[2]

In [4]:
import numpy as np 
# Prune based on 
# Case 2: the pixels projected into image b are outside FOV
# u2_vec bounds should be: 0 image_width
# v2_vec bounds should be: 0, image_height
## do u2-based pruning

image_width = 640
image_height = 480

u2_vec_lower_bound = 0.0
epsilon = 1e-3
u2_vec_upper_bound = image_height*1.0-epsilon

v2_vec_lower_bound = 0.0
v2_vec_upper_bound = image_width*1.0-epsilon

#lower_bound_vec = np.ones_like(u2_vec)* u2_vec_lower_bound
#upper_bound_vec = np.ones_like(u2_vec)* u2_vec_upper_bound
#zeros_vec = np.ones_like(u2_vec)

u2_bound_indices = np.where((u2_vec>u2_vec_lower_bound)&(u2_vec<u2_vec_upper_bound),1,0)
v2_bound_indices = np.where((v2_vec>v2_vec_lower_bound)&(v2_vec<v2_vec_upper_bound),1,0)
in_bound_indices = u2_bound_indices&v2_bound_indices
in_indices = np.where(in_bound_indices>0)

u2_vec_prune = u2_vec[in_indices]
v2_vec_prune = v2_vec[in_indices]
z2_vec_prune = z2_vec[in_indices]
#u_vec_prune = u_vec[in_indices]
#v_vec_prune = v_vec[in_indices]
#z_vec_prune = z_vec_float[in_indices]

# Prune based on 
# Case 3: the pixels in image b are occluded, OR there is no depth return in image b so we aren't sure


u2_vec_long = u2_vec_prune.astype(np.int32)
v2_vec_long = v2_vec_prune.astype(np.int32)
uv2_vec_long = (u2_vec_long, v2_vec_long)


from PIL import Image

img_b_depth = Image.open('/Users/johnny/Code/openself3d/data/2018-04-16-14-25-19/processed/images/000163_depth.png')
img_b_depth = np.asarray(img_b_depth)

img_b_depth_float = img_b_depth.astype(np.float)
depth2_vec = img_b_depth_float[uv2_vec_long]*1.0/DEPTH_IM_SCALE

occlusion_margin = 0.003
z2_vec_prune = z2_vec_prune - occlusion_margin

depth2_01 = np.where((depth2_vec>0)&(depth2_vec>z2_vec_prune),1,0)
non_occluded_indices = np.where(depth2_01>0)
depth2_vec_prune = depth2_vec[non_occluded_indices]

u2_vec_long_prune = u2_vec_long[non_occluded_indices]
v2_vec_long_prune = v2_vec_long[non_occluded_indices]


In [ ]:
import numpy as np

a = np.array([[1,0,0],[0,1,0],[0,0,1],[0,0,0]])
b = np.ones_like(a[0,:]).reshape(1,-1)
print(a.shape)
print(b.shape)
c = np.hstack((a,b))
print(c)

In [ ]:
import correspondence_finder
import correspondence_plotter
import time 
from spartan import SceneStructure

K = np.array([[1,0,0],
              [0,1,0],
              [0,0,1]
              ])
K[0,0] = 533.6422696034836 # focal x
K[1,1] = 534.7824445233571 # focal y
K[0,2] = 319.4091030774892 # principal point x
K[1,2] = 236.4374299691866 # principal point y
K[2,2] = 1.0


scenes_dir ="/Users/johnny/Code/openself3d/data"
sceneStruct= SceneStructure(scenes_dir)
scene = "2018-04-16-14-25-19"
img_a_idx = sceneStruct.get_random_image_index(scene)
img_a_rgb, img_a_depth, img_a_mask, img_a_pose = sceneStruct.get_rgbd_mask_pose(scene, img_a_idx)

img_b_idx = sceneStruct.get_img_idx_with_different_pose(scene, img_a_pose, num_attempts=50)
img_b_rgb, img_b_depth, img_b_mask, img_b_pose = sceneStruct.get_rgbd_mask_pose(scene, img_b_idx)

img_a_depth_numpy = np.asarray(img_a_depth)
img_b_depth_numpy = np.asarray(img_b_depth)

start = time.time()
uv_a = (300, 200)
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth_numpy, img_a_pose,
                                                                    img_b_depth_numpy, img_b_pose,
                                                                
                                                                    )
print(time.time()-start, "seconds")
if uv_a is not None:
    correspondence_plotter.plot_correspondences_direct(img_a_rgb, img_a_depth_numpy, img_b_rgb, img_b_depth_numpy, uv_a,uv_b)
else:
    print("try running this cell again, did not find a correspondence for this pixel")





